In [43]:
pwd

'/home/wsuser/work'

In [44]:
!pip install imutils

In [45]:
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from imutils import build_montages
from skimage import feature
from imutils import paths
import os
import pickle

In [46]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='e3RNwCZT4kKfozN7IUrvzSTXDTys6_WGmRWkRp5wrnXf',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'parkinsondiseasedetection-donotdelete-pr-p72otmmg4p1cir'
object_key = 'spiral.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [47]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [48]:
# define the path to the training and testing directories
trainingPath = r"/home/wsuser/work/spiral/training"
testingPath = r"/home/wsuser/work/spiral/testing"

def quantify_image(image):
    # compute the histogram of oriented gradients feature vector for
    # the input image
    features = feature.hog(image, orientations=9, pixels_per_cell=(10, 10), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1")
    # return the feature vector 
    return features



def load_split(path):
    # grab the list of images in the input directory, then initialize
    # the list of data (i.e., images) and class labels 
    imagePaths = list(paths.list_images (path))
    data = [] 
    labels = []
    # loop over the image paths 
    for imagePath in imagePaths:
    # extract the class label from the filename 
        label = imagePath.split(os.path.sep)[-2]
        # load the input image, convert it to grayscale, and resize # it to 200x200 pixels, ignoring aspect ratio
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
        image = cv2.resize(image, (200, 200))
        # threshold the image such that the drawing appears as white
        # on a black background
        image = cv2.threshold (image, 0, 255, 
                               cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        # quantify the image
        features = quantify_image(image)
        # update the data and labels lists, respectively
        data.append(features)
        labels.append(label)
        # return the data and labels
    return (np.array(data), np.array(labels))

# loading the training and testing data 
print("[INFO] loading data...")
(X_train,y_train) = load_split(trainingPath)
(X_test,y_test) = load_split(testingPath)


[INFO] loading data...


In [49]:
# encode the labels as integers
le = LabelEncoder()
y_train =le.fit_transform (y_train)
y_test = le.transform (y_test)
print (X_train. shape, y_train.shape)

(72, 12996) (72,)


In [50]:
print("[INFO] training model") 
model = RandomForestClassifier(n_estimators=100) 
model.fit(X_train, y_train)

[INFO] training model


RandomForestClassifier()

In [51]:
# randomly select a few images and then initialize the output images # for the montage
testingPaths = list(paths.list_images(testingPath))
idxs = np.arange(0, len(testingPaths))
idxs = np.random.choice(idxs, size=(25,), replace=False)
images=[]

In [52]:
# loop over the testing samples 
for i in idxs:
    # load the testing image, clone it, and resize it
    image = cv2.imread(testingPaths[i])
    output = image.copy()
    output = cv2.resize(output, (128, 128))
    # pre-process the image in the same manner we did earlier
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (200, 200))
    image = cv2.threshold (image, 0, 255,cv2.THRESH_BINARY_INV | cv2. THRESH_OTSU) [1]

In [53]:
# quantify the image and make predictions based on the extracted # features using the last trained Random Forest 
features = quantify_image(image)
preds = model.predict([features])
label =le.inverse_transform(preds)[0]
# draw the colored class label on the output image and add it to # the set of output images
color = (0, 255, 0) if label == "healthy" else (0, 0, 255)
cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
images.append(output)

In [54]:
montage = build_montages(images, (128, 128), (5, 5))[0]

In [55]:
#cv2.imshow("Output", montage) 
#cv2.waitKey(0)

In [56]:
# make predictions on the testing data
predictions = model.predict(X_test) 
# compute the confusion matrix and and use it to derive the raw
# accuracy
cm = confusion_matrix(y_test, predictions).flatten() 
print(cm)
(tn, fp, fn, tp) = cm
accuracy = (tp + tn) / float(cm.sum())
print (accuracy)

[13  2  2 13]
0.8666666666666667


In [57]:
pickle.dump(model, open ('parkinson.pkl', 'wb'))

In [58]:
!tar -zcvf parkinsonsmodel.tgz parkinson.pkl

parkinson.pkl


In [59]:
ls

parkinson.pkl  parkinsonsmodel.tgz  spiral/


In [60]:
!pip install watson-machine-learning-client --upgrade

In [61]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url" : "https://us-south.ml.cloud.ibm.com",
                    "apikey" : "_2x2oURy4R5_OGd7na7vxpRgIimaJ1qKb7LlyeJQfqvD"
                    }
client = APIClient(wml_credentials)

In [62]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    print(space)
    return (next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [63]:
space_uid = guid_from_space_name(client,'ParkinsonsDiseaseDetection')
print('Space UID = ' + space_uid)

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/64107ff3d7984d8f9cfb43723d131694:4f345776-bd04-4e96-9f96-9360a5c1561b::', 'guid': '4f345776-bd04-4e96-9f96-9360a5c1561b', 'name': 'Watson Machine Learning-tw', 'type': 'machine_learning'}], 'description': '', 'name': 'ParkinsonsDiseaseDetection', 'scope': {'bss_account_id': '64107ff3d7984d8f9cfb43723d131694'}, 'stage': {'production': False}, 'status': {'state': 'active'}, 'storage': {'properties': {'bucket_name': '5169496a-049f-41b0-87f1-ce4220bce213', 'bucket_region': 'us-south', 'credentials': {'admin': {'access_key_id': '4fb4c5e666dd47e0a868ee311100107e', 'api_key': 'HNaAl0WvBd2cWscYhahLa6qdgi2SXIlN_TFm397ESiRF', 'secret_access_key': '368eb24a564b620461e99bc26c9697470a0158deb90d525b', 'service_id': 'ServiceId-3baef5fc-695a-4fe1-be52-39d7dc6a8974'}, 'editor': {'access_key_id': 'd144c147db4442ffb6b0f874c16698b9', 'api_key': '4X0x1N2hr8MATzL4-0ophaQSD6_xv48RM71_A5Whjscb', 'resource_key_crn': 'crn:v1:

In [64]:
client.set.default_space(space_uid)

'SUCCESS'

In [65]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [66]:
software_spec_uid=client.software_specifications.get_uid_by_name("default_py3.6") 
software_spec_uid

'65e171d7-72d1-55d9-8ebb-f813d620c9bb'

In [ ]:
model_details = client.repository.store_model(model='parkinsonsmodel.tgz',meta_props={
client.repository.ModelMetaNames.NAME: "parkinson", 
client.repository.ModelMetaNames.TYPE: "default_py3.6", 
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid})
model_id = client.repository.get_model_uid(model_details)